## Projeto CriaComp

## Bibliotecas e Instalações

In [ ]:
# Passo 0: Instalar bibliotecas e baixar o modelo de forma OTIMIZADA

# 1. Bibliotecas para a interface e visualização
!pip install -q gradio supervision

# 2. Biblioteca principal do Segment Anything 2 (SAM)
!pip install -q git+https://github.com/facebookresearch/segment-anything-2.git

# 3. Bibliotecas para a comparação de texto (Prática Cognitiva)
!pip install -q fuzzywuzzy python-Levenshtein

# 4. Bibliotecas para a comparação de texto (OpenAI)
!pip install -q openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Obtendo a chave da OpenAI
from google.colab import userdata
userdata.get('OPENAI_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
# Importando bibliotecas importantes
import os
import zipfile
import json
import openai

In [ ]:
# Define o caminho do arquivo do modelo
checkpoint_path = "checkpoints/sam2_hiera_large.pt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Cria o diretório se não existir
os.makedirs(checkpoint_dir, exist_ok=True)

# Verifica se o modelo já foi baixado e se é válido pelo tamanho
file_is_valid = False
if os.path.exists(checkpoint_path):
    # Um arquivo válido tem mais de 2GB. Um arquivo menor está corrompido.
    try:
        file_size_gb = os.path.getsize(checkpoint_path) / (1024**3)
        if file_size_gb > 2.0:
            print(f"Modelo encontrado com tamanho válido ({file_size_gb:.2f} GB). Download pulado.")
            file_is_valid = True
        else:
            print(f"Modelo encontrado, mas o tamanho ({file_size_gb:.2f} GB) é muito pequeno. Removendo arquivo corrompido.")
            os.remove(checkpoint_path) # Remove o arquivo inválido
    except OSError as e:
        print(f"Erro ao acessar o arquivo: {e}. Removendo para tentar novamente.")
        os.remove(checkpoint_path)

if not file_is_valid:
    print(f"Iniciando download do modelo para '{checkpoint_path}'...")
    # Baixa o modelo mostrando o progresso
    !wget https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt -O {checkpoint_path}
    print("Download concluído.")

Modelo encontrado, mas o tamanho (0.84 GB) é muito pequeno. Removendo arquivo corrompido.
Iniciando download do modelo para 'checkpoints/sam2_hiera_large.pt'...
--2025-08-13 16:49:07--  https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.108, 3.163.189.96, 3.163.189.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 897952466 (856M) [application/vnd.snesdev-page-table]
Saving to: ‘checkpoints/sam2_hiera_large.pt’

checkpoints/sam2_hi 100%[===================>] 856.35M  88.0MB/s    in 34s     

2025-08-13 16:49:42 (24.9 MB/s) - ‘checkpoints/sam2_hiera_large.pt’ saved [897952466/897952466]

Download concluído.


In [ ]:
# Outras bibliotecas importantes
import gradio as gr
import torch
import numpy as np
from PIL import Image
import shutil
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

## Configuração do Modelo

In [ ]:
# --- CONFIGURAÇÃO DO MODELO (Executado apenas uma vez) ---
print("Carregando o modelo SAM 2. Isso pode levar um momento...")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CHECKPOINT_PATH = "checkpoints/sam2_hiera_large.pt"
CONFIG_PATH = "sam2_hiera_l.yaml"

# Verifica se o arquivo de configuração existe, se não, cria um básico.
if not os.path.exists(CONFIG_PATH):
    with open(CONFIG_PATH, 'w') as f:
        f.write("# Arquivo de configuração para SAM 2\n")

sam2_model = build_sam2(CONFIG_PATH, CHECKPOINT_PATH, device=DEVICE)

# O gerador de máscaras é a principal ferramenta para segmentação automática.
mask_generator = SAM2AutomaticMaskGenerator(
    model=sam2_model, #modelo usado
    points_per_side=32, #Mais pontos = mais precisão, mas também maior custo computacional e maior detalhamento
    pred_iou_thresh=0.86, #mínimo de qualidade para manter uma máscara gerada
    stability_score_thresh=0.90, #filtram máscaras ruins
    crop_n_layers=0, #melhora a qualidade com múltiplas escalas, mas 0 é mais rápido
    min_mask_region_area=2000 #remove ruídos pequenos
)
print("Modelo carregado com sucesso!")


Carregando o modelo SAM 2. Isso pode levar um momento...
Modelo carregado com sucesso!


## Criação da Aplicação

In [ ]:
# --- ARMAZENAMENTO DAS ANOTAÇÕES E DADOS GLOBAIS ---
ANNOTATIONS = {}
MASKS_CACHE = {}
TEMP_DIR = "segmentos_temp"

# --- LÓGICA DA APLICAÇÃO ---

if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
os.makedirs(TEMP_DIR, exist_ok=True)

def segment_image(image_pil):
    global ANNOTATIONS, MASKS_CACHE
    if image_pil is None:
        return [], None, None, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    print("Iniciando segmentação...")
    ANNOTATIONS.clear()
    MASKS_CACHE.clear()
    if os.path.exists(TEMP_DIR): shutil.rmtree(TEMP_DIR)
    os.makedirs(TEMP_DIR, exist_ok=True)

    image_rgb = image_pil.convert("RGB")
    masks = mask_generator.generate(np.array(image_rgb))

    if not masks:
        return [], None, None, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    sorted_masks = sorted(masks, key=lambda x: x['area'], reverse=True)
    image_rgba_np = np.array(image_pil.convert("RGBA"))
    segment_paths = []

    for i, mask_data in enumerate(sorted_masks):
        filename = f"segmento_{i}.png"
        filepath = os.path.join(TEMP_DIR, filename)
        MASKS_CACHE[filename] = mask_data['segmentation']
        segment_img_np = image_rgba_np.copy()
        segment_img_np[~mask_data['segmentation']] = [0, 0, 0, 0]
        coords = np.argwhere(mask_data['segmentation'])
        y0, x0 = coords.min(axis=0)
        y1, x1 = coords.max(axis=0) + 1
        cropped_segment = segment_img_np[y0:y1, x0:x1]
        Image.fromarray(cropped_segment).save(filepath)
        segment_paths.append(filepath)

    return segment_paths, segment_paths, image_pil, gr.update(visible=True), gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)

def update_overlay_image(original_image_pil):
    if original_image_pil is None: return None
    overlay_img = np.array(original_image_pil.convert("RGB"))
    for filename, data in ANNOTATIONS.items():
        if data.get("valid") and filename in MASKS_CACHE:
            mask = MASKS_CACHE[filename]
            color = np.random.randint(0, 255, 3)
            overlay_img[mask] = overlay_img[mask] * 0.5 + color * 0.5
    return Image.fromarray(overlay_img.astype(np.uint8))

def get_annotation_data(evt: gr.SelectData, all_paths):
    selected_path = all_paths[evt.index]
    filename = os.path.basename(selected_path)
    existing_annotation = ANNOTATIONS.get(filename, {"valid": False, "description": ""})
    is_valid = existing_annotation["valid"]
    return selected_path, filename, filename, is_valid, existing_annotation["description"], gr.update(visible=is_valid)

def handle_valid_checkbox(is_valid, filename, original_image):
    global ANNOTATIONS
    if not filename: return gr.update(visible=False), "Selecione um segmento primeiro.", original_image
    if is_valid: return gr.update(visible=True), "Adicione uma descrição e salve.", original_image
    else:
        status_message = f"Segmento '{filename}' descartado." if filename in ANNOTATIONS else "Segmento não estava salvo."
        if filename in ANNOTATIONS: del ANNOTATIONS[filename]
        return gr.update(visible=False), status_message, update_overlay_image(original_image)

def save_annotation(original_filename, new_filename_text, description, original_image):
    global ANNOTATIONS, MASKS_CACHE
    if not original_filename: return "Erro: Nenhum segmento selecionado.", None, None, original_image
    new_filename = new_filename_text if new_filename_text.lower().endswith('.png') else new_filename_text + '.png'
    if original_filename != new_filename:
        original_path, new_path = os.path.join(TEMP_DIR, original_filename), os.path.join(TEMP_DIR, new_filename)
        if os.path.exists(new_path): return f"Erro: O nome de arquivo '{new_filename}' já existe.", None, None, original_image
        try:
            os.rename(original_path, new_path)
            if original_filename in ANNOTATIONS: del ANNOTATIONS[original_filename]
            if original_filename in MASKS_CACHE: MASKS_CACHE[new_filename] = MASKS_CACHE.pop(original_filename)
        except OSError as e: return f"Erro ao renomear arquivo: {e}", None, None, original_image
    ANNOTATIONS[new_filename] = {"valid": True, "description": description}
    updated_paths = sorted([os.path.join(TEMP_DIR, f) for f in os.listdir(TEMP_DIR)])
    return f"Anotação salva para '{new_filename}'!", updated_paths, updated_paths, update_overlay_image(original_image)

def finalize_annotations():
    global MASKS_CACHE
    all_files_on_disk = os.listdir(TEMP_DIR)
    valid_filenames = list(ANNOTATIONS.keys())
    deleted_count = 0
    for filename in all_files_on_disk:
        if filename not in valid_filenames:
            try:
                os.remove(os.path.join(TEMP_DIR, filename))
                if filename in MASKS_CACHE: del MASKS_CACHE[filename]
                deleted_count += 1
            except OSError as e: print(f"Erro ao deletar arquivo inválido {filename}: {e}")
    status = f"Finalizado! {deleted_count} segmentos inválidos foram apagados."
    return gr.update(visible=True), gr.update(visible=True), gr.update(choices=valid_filenames, value=None), gr.update(choices=valid_filenames, value=None), status

def export_annotations_to_json():
    json_data = {fn: data["description"] for fn, data in ANNOTATIONS.items() if data.get("valid")}
    if not json_data: return None, None
    json_path = "annotations.json"
    with open(json_path, 'w', encoding='utf-8') as f: json.dump(json_data, f, ensure_ascii=False, indent=4)
    return json_path, json_data

def show_and_get_description(filename):
    if not filename: return None, "Nenhuma descrição."
    image_path = os.path.join(TEMP_DIR, filename)
    description = ANNOTATIONS.get(filename, {}).get("description", "Nenhuma descrição válida encontrada.")
    return image_path, f"Descrição: {description}"

def prepare_download_zip():
    valid_paths = [os.path.join(TEMP_DIR, fn) for fn in ANNOTATIONS if ANNOTATIONS[fn].get("valid")]
    if not valid_paths: return None
    zip_path = "segmentos_validos.zip"
    with zipfile.ZipFile(zip_path, 'w') as zf:
        for p in valid_paths: zf.write(p, os.path.basename(p))
    return zip_path

def compare_descriptions_openai(practice_filename, practice_description, correct_annotations):
    # --- CORREÇÃO APLICADA AQUI ---
    global OPENAI_API_KEY
    if not practice_filename or not practice_description:
        return "Por favor, escolha uma imagem e digite uma descrição."
    if not correct_annotations:
        return "Por favor, exporte o arquivo JSON na primeira aba para carregar as respostas."
    if not OPENAI_API_KEY or OPENAI_API_KEY == "COLE_SUA_CHAVE_DA_API_AQUI":
        return "ERRO: A chave da API da OpenAI não foi definida no código fonte."

    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    original_description = correct_annotations.get(practice_filename)
    if not original_description:
        return f"Não foi encontrada uma anotação para '{practice_filename}' no arquivo JSON."

    prompt = f"""
    Você é um assistente avaliador. Compare as duas descrições de uma imagem a seguir, focando no significado semântico.
    Descrição Original: "{original_description}"
    Descrição do Usuário: "{practice_description}"

    Responda em duas linhas:
    1. Na primeira linha, forneça um placar de semelhança de 0 a 100.
    2. Na segunda linha, escreva um feedback conciso e amigável em português sobre a comparação.

    Exemplo de Resposta:
    Placar: 95
    Feedback: Excelente! As descrições são semanticamente quase idênticas.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=100
        )
        ai_response = response.choices[0].message.content
        return f"**Resultado da Comparação (IA):**\n\n{ai_response}\n\n**Descrição Original:**\n'{original_description}'"
    except openai.AuthenticationError:
        return "ERRO: A chave da API da OpenAI é inválida ou expirou."
    except Exception as e:
        return f"ERRO: Ocorreu um erro ao contatar a API da OpenAI: {e}"

# --- CONSTRUÇÃO DA INTERFACE ---
with gr.Blocks(theme=gr.themes.Soft(), css="footer {display: none !important}") as demo:
    gr.Markdown("# 🖼️ Ferramenta de Anotação e Prática Cognitiva com SAM 2")
    segment_paths_state, original_image_state, json_content_state = gr.State([]), gr.State(), gr.State()

    with gr.Tabs() as tabs:
        with gr.TabItem("1. Segmentar e Anotar", id=0) as annotate_tab:
            with gr.Row():
                with gr.Column(scale=1):
                    input_image = gr.Image(type="pil", label="Carregue sua Imagem")
                    segment_button = gr.Button("▶️ Segmentar Imagem", variant="primary")
                    overlay_image_display = gr.Image(label="Visualização das Máscaras Válidas", interactive=False)
                with gr.Column(scale=2):
                    gallery = gr.Gallery(label="Segmentos Gerados", elem_id="gallery", columns=6, height="auto")
            with gr.Group(visible=False) as annotation_box:
                with gr.Row():
                    selected_image_preview = gr.Image(label="Segmento Selecionado", interactive=False)
                    with gr.Column():
                        original_filename_hidden, editable_filename_textbox = gr.Textbox(visible=False), gr.Textbox(label="Nome do Arquivo", interactive=True)
                        is_valid_checkbox = gr.Checkbox(label="Este segmento é válido?")
                        with gr.Group(visible=False) as annotation_fields:
                            description_textbox = gr.Textbox(label="Descrição", lines=3)
                            save_button = gr.Button("💾 Salvar Anotação", variant="secondary")
            status_textbox = gr.Textbox(label="Status", interactive=False)
            with gr.Group(visible=False) as download_box:
                gr.Markdown("---")
                gr.Markdown("### Finalizar e Baixar")
                finalize_button = gr.Button("✅ Finalizar Anotação", variant="stop")
                with gr.Row():
                    prepare_download_button, export_json_button = gr.Button("📦 Baixar Imagens (.zip)"), gr.Button("📄 Baixar Anotações (.json)")
                with gr.Row():
                    download_zip_output, download_json_output = gr.File(label=".zip"), gr.File(label=".json")

        with gr.TabItem("2. Consultar Anotações", id=1, visible=False) as query_tab:
            gr.Markdown("### Consultar Descrição de um Segmento")
            gr.Markdown("Escolha um dos segmentos finalizados para ver sua descrição.")
            with gr.Row():
                query_image_selector = gr.Dropdown(label="Escolha um Segmento para Consultar")
                with gr.Column():
                    query_image_preview, query_result = gr.Image(label="Segmento"), gr.Textbox(label="Resultado")

        with gr.TabItem("3. Praticar", id=2, visible=False) as practice_tab:
            gr.Markdown("### Praticar Memória e Cognição com IA")
            gr.Markdown("Escolha um segmento e teste sua memória. A comparação será feita pela IA da OpenAI.")
            with gr.Row():
                with gr.Column():
                    practice_image_selector = gr.Dropdown(label="Escolha um Segmento para Praticar")
                    practice_image_display = gr.Image(label="Imagem Selecionada", interactive=False)
                with gr.Column():
                    # O campo da chave foi removido da interface
                    practice_description_input = gr.Textbox(label="Descreva a imagem de memória", lines=4)
                    compare_button = gr.Button("🧠 Comparar com IA", variant="primary")
                    comparison_result_display = gr.Markdown(label="Resultado da Comparação")

    # --- LÓGICA DOS EVENTOS ---
    segment_button.click(fn=segment_image, inputs=input_image, outputs=[gallery, segment_paths_state, original_image_state, annotation_box, download_box, query_tab, practice_tab]).then(fn=lambda x: x, inputs=input_image, outputs=overlay_image_display)
    gallery.select(fn=get_annotation_data, inputs=[segment_paths_state], outputs=[selected_image_preview, original_filename_hidden, editable_filename_textbox, is_valid_checkbox, description_textbox, annotation_fields])
    is_valid_checkbox.change(fn=handle_valid_checkbox, inputs=[is_valid_checkbox, original_filename_hidden, original_image_state], outputs=[annotation_fields, status_textbox, overlay_image_display])
    save_button.click(fn=save_annotation, inputs=[original_filename_hidden, editable_filename_textbox, description_textbox, original_image_state], outputs=[status_textbox, gallery, segment_paths_state, overlay_image_display])
    finalize_button.click(fn=finalize_annotations, inputs=None, outputs=[query_tab, practice_tab, practice_image_selector, query_image_selector, status_textbox])

    query_image_selector.change(fn=show_and_get_description, inputs=query_image_selector, outputs=[query_image_preview, query_result])
    practice_image_selector.change(fn=lambda x: os.path.join(TEMP_DIR, x) if x else None, inputs=practice_image_selector, outputs=practice_image_display)

    prepare_download_button.click(fn=prepare_download_zip, inputs=None, outputs=download_zip_output)
    export_json_button.click(fn=export_annotations_to_json, inputs=None, outputs=[download_json_output, json_content_state])
    # A chave da API não é mais passada como um input da interface
    compare_button.click(fn=compare_descriptions_openai, inputs=[practice_image_selector, practice_description_input, json_content_state], outputs=comparison_result_display)

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cecd9a3791c7615c2c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Iniciando segmentação...


/usr/local/lib/python3.11/dist-packages/sam2/sam2_image_predictor.py:431: UserWarning: /usr/local/lib/python3.11/dist-packages/sam2/_C.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  masks = self._transforms.postprocess_masks(


Iniciando segmentação...


/usr/local/lib/python3.11/dist-packages/sam2/sam2_image_predictor.py:431: UserWarning: /usr/local/lib/python3.11/dist-packages/sam2/_C.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  masks = self._transforms.postprocess_masks(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://cecd9a3791c7615c2c.gradio.live
